In [1]:
import socket
import dns.resolver

for rdata in dns.resolver.resolve('seed.bitcoin.sipa.be', 'A') :
    print(rdata)

8.209.71.217
167.114.210.39
212.159.24.73
8.129.83.82
142.177.159.125
185.148.145.74
35.185.180.184
8.217.115.93
185.233.189.210
195.99.208.66
187.139.39.207
173.177.237.29
78.46.189.221
8.129.184.255
173.56.235.225
71.183.251.75
35.224.196.150
5.79.123.3
95.216.238.126
47.180.49.158
71.38.122.113
195.201.95.119
157.230.1.157
3.127.236.227
165.22.154.213


https://github.com/bitcoin/bitcoin/tree/master/contrib/seeds


In [ ]:
# curl https://bitcoin.sipa.be/seeds.txt.gz | gzip -dc > seeds_main.txt
import subprocess

with open("seeds_main.txt","w") as out:
    curl_process = subprocess.Popen(['curl', 'https://bitcoin.sipa.be/seeds.txt.gz'],
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE)

    gzip_process = subprocess.Popen(['gzip', '-dc'],
                         stdin=curl_process.stdout,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE)

    curl_process.stdout.close()
    stdout, stderr = gzip_process.communicate()
    gzip_process.wait()
    out.write(stdout.decode("utf-8"))

In [2]:
#!/usr/bin/env python3
# Copyright (c) 2013-2020 The Bitcoin Core developers
# Distributed under the MIT software license, see the accompanying
# file COPYING or http://www.opensource.org/licenses/mit-license.php.
#
# Generate seeds.txt from Pieter's DNS seeder
#

import re
import sys
import dns.resolver
import collections
from contextlib import redirect_stdout

NSEEDS=512

MAX_SEEDS_PER_ASN=2

MIN_BLOCKS = 337600

# These are hosts that have been observed to be behaving strangely (e.g.
# aggressively connecting to every node).
with open("suspicious_hosts.txt", mode="r", encoding="utf-8") as f:
    SUSPICIOUS_HOSTS = {s.strip() for s in f if s.strip()}


PATTERN_IPV4 = re.compile(r"^((\d{1,3})\.(\d{1,3})\.(\d{1,3})\.(\d{1,3})):(\d+)$")
PATTERN_IPV6 = re.compile(r"^\[([0-9a-z:]+)\]:(\d+)$")
PATTERN_ONION = re.compile(r"^([a-z2-7]{56}\.onion):(\d+)$")
PATTERN_AGENT = re.compile(
    r"^/Satoshi:("
    r"0.14.(0|1|2|3|99)|"
    r"0.15.(0|1|2|99)|"
    r"0.16.(0|1|2|3|99)|"
    r"0.17.(0|0.1|1|2|99)|"
    r"0.18.(0|1|99)|"
    r"0.19.(0|1|2|99)|"
    r"0.20.(0|1|2|99)|"
    r"0.21.(0|1|2|99)|"
    r"22.(0|99)|"
    r"23.99"
    r")")

def parseline(line):
    sline = line.split()
    if len(sline) < 11:
       return None
    m = PATTERN_IPV4.match(sline[0])
    sortkey = None
    ip = None
    if m is None:
        m = PATTERN_IPV6.match(sline[0])
        if m is None:
            m = PATTERN_ONION.match(sline[0])
            if m is None:
                return None
            else:
                net = 'onion'
                ipstr = sortkey = m.group(1)
                port = int(m.group(2))
        else:
            net = 'ipv6'
            if m.group(1) in ['::']: # Not interested in localhost
                return None
            ipstr = m.group(1)
            sortkey = ipstr # XXX parse IPv6 into number, could use name_to_ipv6 from generate-seeds
            port = int(m.group(2))
    else:
        # Do IPv4 sanity check
        ip = 0
        for i in range(0,4):
            if int(m.group(i+2)) < 0 or int(m.group(i+2)) > 255:
                return None
            ip = ip + (int(m.group(i+2)) << (8*(3-i)))
        if ip == 0:
            return None
        net = 'ipv4'
        sortkey = ip
        ipstr = m.group(1)
        port = int(m.group(6))
    # Skip bad results.
    if sline[1] == 0:
        return None
    # Extract uptime %.
    uptime30 = float(sline[7][:-1])
    # Extract Unix timestamp of last success.
    lastsuccess = int(sline[2])
    # Extract protocol version.
    version = int(sline[10])
    # Extract user agent.
    agent = sline[11][1:-1]
    # Extract service flags.
    service = int(sline[9], 16)
    # Extract blocks.
    blocks = int(sline[8])
    # Construct result.
    return {
        'net': net,
        'ip': ipstr,
        'port': port,
        'ipnum': ip,
        'uptime': uptime30,
        'lastsuccess': lastsuccess,
        'version': version,
        'agent': agent,
        'service': service,
        'blocks': blocks,
        'sortkey': sortkey,
    }

def dedup(ips):
    '''deduplicate by address,port'''
    d = {}
    for ip in ips:
        d[ip['ip'],ip['port']] = ip
    return list(d.values())

def filtermultiport(ips):
    '''Filter out hosts with more nodes per IP'''
    hist = collections.defaultdict(list)
    for ip in ips:
        hist[ip['sortkey']].append(ip)
    return [value[0] for (key,value) in list(hist.items()) if len(value)==1]

def lookup_asn(net, ip):
    '''
    Look up the asn for an IP (4 or 6) address by querying cymru.com, or None
    if it could not be found.
    '''
    try:
        if net == 'ipv4':
            ipaddr = ip
            prefix = '.origin'
        else:                  # http://www.team-cymru.com/IP-ASN-mapping.html
            res = str()                         # 2001:4860:b002:23::68
            for nb in ip.split(':')[:4]:  # pick the first 4 nibbles
                for c in nb.zfill(4):           # right padded with '0'
                    res += c + '.'              # 2001 4860 b002 0023
            ipaddr = res.rstrip('.')            # 2.0.0.1.4.8.6.0.b.0.0.2.0.0.2.3
            prefix = '.origin6'

        asn = int([x.to_text() for x in dns.resolver.resolve('.'.join(
                   reversed(ipaddr.split('.'))) + prefix + '.asn.cymru.com',
                   'TXT').response.answer][0].split('\"')[1].split(' ')[0])
        return asn
    except Exception as e:
        sys.stderr.write(f'ERR: Could not resolve ASN for "{ip}": {e}\n')
        return None

# Based on Greg Maxwell's seed_filter.py
def filterbyasn(ips, max_per_asn, max_per_net):
    # Sift out ips by type
    ips_ipv46 = [ip for ip in ips if ip['net'] in ['ipv4', 'ipv6']]
    ips_onion = [ip for ip in ips if ip['net'] == 'onion']

    # Filter IPv46 by ASN, and limit to max_per_net per network
    result = []
    net_count = collections.defaultdict(int)
    asn_count = collections.defaultdict(int)
    for ip in ips_ipv46:
        if net_count[ip['net']] == max_per_net:
            continue
        asn = lookup_asn(ip['net'], ip['ip'])
        if asn is None or asn_count[asn] == max_per_asn:
            continue
        asn_count[asn] += 1
        net_count[ip['net']] += 1
        result.append(ip)

    # Add back Onions (up to max_per_net)
    result.extend(ips_onion[0:max_per_net])
    return result

def ip_stats(ips):
    hist = collections.defaultdict(int)
    for ip in ips:
        if ip is not None:
            hist[ip['net']] += 1

    return '%6d %6d %6d' % (hist['ipv4'], hist['ipv6'], hist['onion'])

def make_seeds():
    with open("seeds_main.txt","r") as seeds_main:
        with open('nodes_main.txt', 'w') as f:
            with redirect_stdout(f):
                lines = seeds_main.readlines()
                ips = [parseline(line) for line in lines]

                print('\x1b[7m  IPv4   IPv6  Onion Pass                                               \x1b[0m', file=sys.stderr)
                print('%s Initial' % (ip_stats(ips)), file=sys.stderr)
                # Skip entries with invalid address.
                ips = [ip for ip in ips if ip is not None]
                print('%s Skip entries with invalid address' % (ip_stats(ips)), file=sys.stderr)
                # Skip duplicates (in case multiple seeds files were concatenated)
                ips = dedup(ips)
                print('%s After removing duplicates' % (ip_stats(ips)), file=sys.stderr)
                # Skip entries from suspicious hosts.
                ips = [ip for ip in ips if ip['ip'] not in SUSPICIOUS_HOSTS]
                print('%s Skip entries from suspicious hosts' % (ip_stats(ips)), file=sys.stderr)
                # Enforce minimal number of blocks.
                ips = [ip for ip in ips if ip['blocks'] >= MIN_BLOCKS]
                print('%s Enforce minimal number of blocks' % (ip_stats(ips)), file=sys.stderr)
                # Require service bit 1.
                ips = [ip for ip in ips if (ip['service'] & 1) == 1]
                print('%s Require service bit 1' % (ip_stats(ips)), file=sys.stderr)
                # Require at least 50% 30-day uptime for clearnet, 10% for onion.
                req_uptime = {
                    'ipv4': 50,
                    'ipv6': 50,
                    'onion': 10,
                }
                ips = [ip for ip in ips if ip['uptime'] > req_uptime[ip['net']]]
                print('%s Require minimum uptime' % (ip_stats(ips)), file=sys.stderr)
                # Require a known and recent user agent.
                ips = [ip for ip in ips if PATTERN_AGENT.match(ip['agent'])]
                print('%s Require a known and recent user agent' % (ip_stats(ips)), file=sys.stderr)
                # Sort by availability (and use last success as tie breaker)
                ips.sort(key=lambda x: (x['uptime'], x['lastsuccess'], x['ip']), reverse=True)
                # Filter out hosts with multiple bitcoin ports, these are likely abusive
                ips = filtermultiport(ips)
                print('%s Filter out hosts with multiple bitcoin ports' % (ip_stats(ips)), file=sys.stderr)
                # Look up ASNs and limit results, both per ASN and globally.
                ips = filterbyasn(ips, MAX_SEEDS_PER_ASN, NSEEDS)
                print('%s Look up ASNs and limit results per ASN and per net' % (ip_stats(ips)), file=sys.stderr)
                # Sort the results by IP address (for deterministic output).
                ips.sort(key=lambda x: (x['net'], x['sortkey']))
                for ip in ips:
                    if ip['net'] == 'ipv6':
                        print('[%s]:%i' % (ip['ip'], ip['port']))
                    else:
                        print('%s:%i' % (ip['ip'], ip['port']))

make_seeds()

  IPv4   IPv6  Onion Pass                                               
469972  72995      0 Initial
469972  72995      0 Skip entries with invalid address
469972  72995      0 After removing duplicates
469971  72995      0 Skip entries from suspicious hosts
165822  65162      0 Enforce minimal number of blocks
160699  63222      0 Require service bit 1
  4831   1378      0 Require minimum uptime
  4287   1054      0 Require a known and recent user agent
  4189   1036      0 Filter out hosts with multiple bitcoin ports
ERR: Could not resolve ASN for "200:419f:63fe:40eb:1d13:9000:74b:ae8b": The DNS query name does not exist: b.e.0.4.e.f.3.6.f.9.1.4.0.0.2.0.origin6.asn.cymru.com.
   512    136      0 Look up ASNs and limit results per ASN and per net


In [ ]:
import time
import json
import requests
from requests.exceptions import HTTPError

def get_location(ips, fields="16577"):
    if not isinstance(ips, list):
        ips = [ips]
    
    
    for ip in ips:
        try:
            response = requests.get(f"http://ip-api.com/json/{ip}?fields={fields}")
            response.raise_for_status()
            print(response.headers)
            # access JSOn content
            jsonResponse = response.json()
            print("Entire JSON response")
            print(jsonResponse)
            print(response.headers["X-Rl"])
            if response.headers["X-Rl"] == '39':
                print(f"Going to sleep for: {response.headers['X-Ttl']} seconds")
                time.sleep(int(response.headers["X-Ttl"]))

        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
        except Exception as err:
            print(f'Other error occurred: {err}')
            


In [ ]:
get_location('8.8.8.8')

In [18]:
import socket
import dns.resolver

# Bitcoin Testnet
for rdata in dns.resolver.resolve('testnet-seed.bitcoin.jonasschnelli.ch', 'A') :
    print(rdata)

5.9.158.123
34.79.158.233
162.55.181.95
104.155.226.24
176.57.189.177
88.198.91.250
3.112.61.112
170.75.165.230
13.250.1.104
142.93.129.235
169.63.212.95
167.179.98.113
165.227.60.15
51.79.82.75
46.242.129.25
176.9.71.156
35.224.245.248
95.217.83.249
75.119.141.144
18.162.45.10
52.37.192.249
18.191.163.39
93.157.187.23


https://developer.bitcoin.org/reference/p2p_networking.html#message-headers

The following example is an annotated hex dump of a mainnet message header from a “verack” message which has no payload.<br>

```
f9beb4d9 ................... Start string: Mainnet <br>
76657261636b000000000000 ... Command name: verack + null padding<br>
00000000 ................... Byte count: 0<br>
5df6e0e2 ................... Checksum: SHA256(SHA256(<empty>))<br>
```

In [26]:
import socket
import struct
import codecs
import time
import random
import hashlib
import binascii

default_port    = 18333
magic_          = 0x0709110B
max_n_bits      = 0x1d00ffff
version         = 70015
command         = b"version"
#host            = "8.9.3.218"
host            = "82.197.160.8"   # node Thush

# Start String

magic               = struct.pack("i",magic_)                    # magic value                           
command             = struct.pack("<12s",command)                # command
version             = struct.pack("i",version)                   # Protocol Version
services            = struct.pack("Q",0)                         # Services: Headers Only (SPV)
timestamp           = struct.pack("q",int(time.time()))          # Timestamp
addr_recv_services  = struct.pack("Q",0);
addr_recv_ip        = struct.pack(">16s",bytes(host, 'utf-8'));  # Receiver IP Address
addr_recv_port      = struct.pack(">H",default_port);            # Receiver Port
addr_trans_services = struct.pack("Q",0);
addr_trans_ip       = struct.pack(">16s",bytes("",'utf-8'))      # Sender IP Address
addr_trans_port     = struct.pack(">H",0)                        # Sender Port
nonce               = struct.pack("Q", random.getrandbits(64))   # Nonce (not used here)
user_agent_byes     = struct.pack("B",0)                         # Bytes in version string
start_height        = struct.pack("i",596306)                    # Starting block height: 329107
relay               = struct.pack("?",False)                     # Relay transactions: false

#)

payload = version + services + timestamp + addr_recv_services + addr_recv_ip + addr_recv_port + addr_trans_services + addr_trans_ip + addr_trans_port + nonce + user_agent_byes + start_height + relay;

length = struct.pack("I", len(payload));
checksum = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[:4];
print(binascii.hexlify(magic + command + length + checksum + payload))
msg =  magic + command + length + checksum + payload


sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM);

sock.connect((host,default_port));
sock.send(msg);
time.sleep(1);
reply= sock.recv(1024);
print(binascii.hexlify(reply))
sock.close()


b'0b11090776657273696f6e000000000056000000b5cf4fbe7f1101000000000000000000823f346200000000000000000000000038322e3139372e3136302e3800000000479d0000000000000000000000000000000000000000000000000000581715f779b11628005219090000'
b'0b11090776657273696f6e0000000000660000003763bc17801101000904000000000000823f346200000000000000000000000000000000000000000000ffff825cff23cd010904000000000000000000000000000000000000000000000000d6ebd6177dfe1e32102f5361746f7368693a32322e302e302fdc6b2100010b11090776657261636b000000000000000000005df6e0e2'


https://developer.bitcoin.org/examples/p2p_networking.html#retrieving-a-merkleblock

# Common structures
https://en.bitcoin.it/wiki/Protocol_documentation#Message_structure

### Message structure
<table class="wikitable">
<tbody><tr>
<th> Field Size </th>
<th> Description </th>
<th> Data type </th>
<th> Comments
</th></tr>
<tr>
<td> 4 </td>
<td> magic </td>
<td> uint32_t </td>
<td> Magic value indicating message origin network, and used to seek to next message when stream state is unknown
</td></tr>
<tr>
<td> 12 </td>
<td> command </td>
<td> char[12] </td>
<td> ASCII string identifying the packet content, NULL padded (non-NULL padding results in packet rejected)
</td></tr>
<tr>
<td> 4 </td>
<td> length </td>
<td> uint32_t </td>
<td> Length of payload in number of bytes
</td></tr>
<tr>
<td> 4 </td>
<td> checksum </td>
<td> uint32_t </td>
<td> First 4 bytes of sha256(sha256(payload))
</td></tr>
<tr>
<td>&nbsp;? </td>
<td> payload </td>
<td> uchar[] </td>
<td> The actual data
</td></tr></tbody></table>

### Magic values

<table class="wikitable">
<tbody><tr>
<th> Network </th>
<th> Magic value </th>
<th> Sent over wire as
</th></tr>
<tr>
<td> main </td>
<td> 0xD9B4BEF9 </td>
<td> F9 BE B4 D9
</td></tr>
<tr>
<td> testnet/regtest </td>
<td> 0xDAB5BFFA </td>
<td> FA BF B5 DA
</td></tr>
<tr>
<td> testnet3 </td>
<td> 0x0709110B </td>
<td> 0B 11 09 07
</td></tr>
<tr>
<td> signet(default) </td>
<td> 0x40CF030A </td>
<td> 0A 03 CF 40
</td></tr>
<tr>
<td> namecoin </td>
<td> 0xFEB4BEF9 </td>
<td> F9 BE B4 FE
</td></tr></tbody></table>

### Variable length integer

<table class="wikitable">
<tbody><tr>
<th> Value </th>
<th> Storage length </th>
<th> Format
</th></tr>
<tr>
<td> &lt; 0xFD </td>
<td> 1 </td>
<td> uint8_t
</td></tr>
<tr>
<td> &lt;= 0xFFFF </td>
<td> 3 </td>
<td> 0xFD followed by the length as uint16_t
</td></tr>
<tr>
<td> &lt;= 0xFFFF FFFF </td>
<td> 5 </td>
<td> 0xFE followed by the length as uint32_t
</td></tr>
<tr>
<td> - </td>
<td> 9 </td>
<td> 0xFF followed by the length as uint64_t
</td></tr></tbody></table>

### Variable length string

<table class="wikitable">
<tbody><tr>
<th> Field Size </th>
<th> Description </th>
<th> Data type </th>
<th> Comments
</th></tr>
<tr>
<td> 1+ </td>
<td> length </td>
<td> <a href="/wiki/Protocol_documentation#Variable_length_integer" title="Protocol documentation">var_int</a> </td>
<td> Length of the string
</td></tr>
<tr>
<td>&nbsp;? </td>
<td> string </td>
<td> char[] </td>
<td> The string itself (can be empty)
</td></tr></tbody></table>


### Network address

<table class="wikitable">
<tbody><tr>
<th> Field Size </th>
<th> Description </th>
<th> Data type </th>
<th> Comments
</th></tr>
<tr>
<td> 4 </td>
<td> time </td>
<td> uint32 </td>
<td> the Time (version &gt;= 31402). <b>Not present in version message.</b>
</td></tr>
<tr>
<td> 8 </td>
<td> services </td>
<td> uint64_t </td>
<td> same service(s) listed in <a href="#version">version</a>
</td></tr>
<tr>
<td> 16 </td>
<td> IPv6/4 </td>
<td> char[16] </td>
<td> IPv6 address. Network byte order. The original client only supported IPv4 and only read the last 4 bytes to get the IPv4 address. However, the IPv4 address is written into the message as a 16 byte <a rel="nofollow" class="external text" href="http://en.wikipedia.org/wiki/IPv6#IPv4-mapped_IPv6_addresses">IPv4-mapped IPv6 address</a>
<p>(12 bytes <i>00 00 00 00 00 00 00 00 00 00 FF FF</i>, followed by the 4 bytes of the IPv4 address).
</p>
</td></tr>
<tr>
<td> 2 </td>
<td> port </td>
<td> uint16_t </td>
<td> port number, network byte order
</td></tr></tbody></table>

```

0000   f9 be b4 d9 76 65 72 73 69 6f 6e 00 00 00 00 00  ....version.....
0010   64 00 00 00 35 8d 49 32 62 ea 00 00 01 00 00 00  d...5.I2b.......
0020   00 00 00 00 11 b2 d0 50 00 00 00 00 01 00 00 00  .......P........
0030   00 00 00 00 00 00 00 00 00 00 00 00 00 00 ff ff  ................
0040   00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
0050   00 00 00 00 00 00 00 00 ff ff 00 00 00 00 00 00  ................
0060   3b 2e b3 5d 8c e6 17 65 0f 2f 53 61 74 6f 73 68  ;..]...e./Satosh
0070   69 3a 30 2e 37 2e 32 2f c0 3e 03 00              i:0.7.2/.>..


Message Header:
 F9 BE B4 D9                                                                   - Main network magic bytes
 76 65 72 73 69 6F 6E 00 00 00 00 00                                           - "version" command
 64 00 00 00                                                                   - Payload is 100 bytes long
 35 8d 49 32                                                                   - payload checksum (internal byte order)

Version message:
 62 EA 00 00                                                                   - 60002 (protocol version 60002)
 01 00 00 00 00 00 00 00                                                       - 1 (NODE_NETWORK services)
 11 B2 D0 50 00 00 00 00                                                       - Tue Dec 18 10:12:33 PST 2012
 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 FF FF 00 00 00 00 00 00 - Recipient address info - see Network Address
 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 FF FF 00 00 00 00 00 00 - Sender address info - see Network Address
 3B 2E B3 5D 8C E6 17 65                                                       - Node ID
 0F 2F 53 61 74 6F 73 68 69 3A 30 2E 37 2E 32 2F                               - "/Satoshi:0.7.2/" sub-version string (string is 15 bytes long)
 C0 3E 03 00                                                                   - Last block sending node has is block #212672
 ```
 
 
f9 be b4 d9 76 65 72 73 69 6f 6e 00 00 00 00 00
64 00 00 00 35 8d 49 32 62 ea 00 00 01 00 00 00
00 00 00 00 11 b2 d0 50 00 00 00 00 01 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 ff ff
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 ff ff 00 00 00 00 00 00
3b 2e b3 5d 8c e6 17 65 0f 2f 53 61 74 6f 73 68
69 3a 30 2e 37 2e 32 2f c0 3e 03 00            

In [4]:
from test_framework.messages import *

In [13]:
message = msg_version()



In [17]:
import io
message.deserialize(io.BytesIO(reply))


In [18]:
message

msg_version(nVersion=118034699 nServices=31084746137298294 nTime=Mon May 31 14:03:12 15852 addrTo=CAddress(nServices=300716652533479 net=IPv4 addr=0.0.0.0 port=0) addrFrom=CAddress(nServices=0 net=IPv4 addr=255.35.204.241 port=2308) nNonce=0x0000000000000000 strSubVer= nStartingHeight=0 relay=0)

In [104]:
from datetime import datetime
dt_object = datetime.fromtimestamp(msg.nTime)

print("dt_object =", dt_object)

AttributeError: 'msg_verack' object has no attribute 'nTime'

In [27]:
time.time()

1647591412.3683789